In [1]:
!pip install git+https://github.com/huggingface/transformers accelerate

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-i4d5nigx
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-i4d5nigx
  Resolved https://github.com/huggingface/transformers to commit 3a6ab46a0b85479d6fb0d6ce0bff2e48b4751ac4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s 

In [4]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 21.6 MB/s eta 0:00:00


In [42]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)


processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
image_path="/content/test_paper_1.jpg"
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": "Give question number and their  questions and thiers max marks in json format if a  question has more than one subpart then give  question as their subpart but max max of combined.Give output in json format"},
        ],
    }
]

In [8]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)


In [9]:
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=1021)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [10]:
# Joining the list into a properly formatted string
formatted_output = "\n".join(output_text).strip()

Qn_paper_json="Extracted Text:\n" + formatted_output

In [11]:
print(Qn_paper_json)

Extracted Text:
```json
{
  "questions": [
    {
      "question_number": "Q1",
      "subparts": [
        {
          "subpart_number": "a",
          "description": "RTL is used to describe"
        },
        {
          "subpart_number": "b",
          "description": "Sign of the number is preserved in shift micro-operation."
        },
        {
          "subpart_number": "c",
          "description": "PC contains the address of"
        },
        {
          "subpart_number": "d",
          "description": "What decisions are made in computer organization?"
        },
        {
          "subpart_number": "e",
          "description": "Convert (568)10->()2->()8->()"
        },
        {
          "subpart_number": "f",
          "description": "Define micro-operation and cite an example of micro-operation."
        },
        {
          "subpart_number": "g",
          "description": "Differentiate fixed point and floating point representation."
        },
        {
          

#Answer_sheet_extraction

In [84]:
image_path="/content/one.jpg"

example="""
{
  "questions": [
    {
      "question_number": "Q1",
      "marks_obtained": 10
    },
    {
      "question_number": "Q2",
      "marks_obtained": 2
    }
  ]
}

"""

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": f"Give question number and their Marks obtained in json format.consider that Question numbsr only which has Obtained marks handwritten,.Do not wite null or anything just laeve that and do not hellusinate.Give output in json format.exampleformat is {example}."},
        ],
    }
]

In [85]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [86]:
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

In [87]:
# Joining the list into a properly formatted string
formatted_output = "\n".join(output_text).strip()
Answer_sheet_json_one="Extracted Text:\n" + formatted_output

In [88]:
print(Answer_sheet_json_one)

Extracted Text:
```json
{
  "questions": [
    {
      "question_number": "Q1",
      "marks_obtained": 22
    },
    {
      "question_number": "Q2",
      "marks_obtained": 23
    },
    {
      "question_number": "Q3",
      "marks_obtained": 24
    },
    {
      "question_number": "Q4",
      "marks_obtained": 25
    },
    {
      "question_number": "Q5",
      "marks_obtained": 26


In [74]:
Answer_sheet_json="Extracted Text:\n" + formatted_output
print(Answer_sheet_json)

Extracted Text:
```json
{
  "questions": [
    {
      "question_number": "Q1",
      "marks_obtained": 10
    },
    {
      "question_number": "Q2",
      "marks_obtained": 2
    },
    {
      "question_number": "Q3",
      "marks_obtained": 3
    },
    {
      "question_number": "Q4",
      "marks_obtained": 0
    }
  ]
}
```


#**merging and taking final output**

In [89]:
print(Qn_paper_json)

Extracted Text:
```json
{
  "questions": [
    {
      "question_number": "Q1",
      "subparts": [
        {
          "subpart_number": "a",
          "description": "RTL is used to describe"
        },
        {
          "subpart_number": "b",
          "description": "Sign of the number is preserved in shift micro-operation."
        },
        {
          "subpart_number": "c",
          "description": "PC contains the address of"
        },
        {
          "subpart_number": "d",
          "description": "What decisions are made in computer organization?"
        },
        {
          "subpart_number": "e",
          "description": "Convert (568)10->()2->()8->()"
        },
        {
          "subpart_number": "f",
          "description": "Define micro-operation and cite an example of micro-operation."
        },
        {
          "subpart_number": "g",
          "description": "Differentiate fixed point and floating point representation."
        },
        {
          

In [90]:
print(Answer_sheet_json)

Extracted Text:
```json
{
  "questions": [
    {
      "question_number": "Q1",
      "marks_obtained": 10
    },
    {
      "question_number": "Q2",
      "marks_obtained": 2
    },
    {
      "question_number": "Q3",
      "marks_obtained": 3
    },
    {
      "question_number": "Q4",
      "marks_obtained": 0
    }
  ]
}
```


**Taking Out Combined Output**

In [91]:
from google.colab import userdata
api_key=userdata.get('GOOGLE_API_KEY')

In [79]:
#few sort prompting for model
outputs="""

{
  "questions": [
    {
      "question_number": "Q1",
      "description": "Instruction execution cycle",
      "max_marks": 5,
      "marks_obtained": 2
    }
  ]
}
"""


prompt_combining_ocr=f"""
  You are given two JSON objects: {Qn_paper_json}` and :{Answer_sheet_json}.

- {Qn_paper_json} contains a list of questions, subparts, descriptions, and maximum marks.
- {Answer_sheet_json}contains the marks obtained for each question.

Your task is to merge these two JSONs and give output only json object.

Return a **single valid JSON object** with all questions, subparts, `"max_marks"`, and `"marks_obtained"`.

yout output will be like {outputs}


"""

In [92]:
model = genai.GenerativeModel("gemini-2.0-flash")
response = model.generate_content(prompt)
print(response.text)
json_obj=response.text

```json
{
  "questions": [
    {
      "question_number": "Q1",
      "subparts": [
        {
          "subpart_number": "a",
          "description": "RTL is used to describe"
        },
        {
          "subpart_number": "b",
          "description": "Sign of the number is preserved in shift micro-operation."
        },
        {
          "subpart_number": "c",
          "description": "PC contains the address of"
        },
        {
          "subpart_number": "d",
          "description": "What decisions are made in computer organization?"
        },
        {
          "subpart_number": "e",
          "description": "Convert (568)10->()2->()8->()"
        },
        {
          "subpart_number": "f",
          "description": "Define micro-operation and cite an example of micro-operation."
        },
        {
          "subpart_number": "g",
          "description": "Differentiate fixed point and floating point representation."
        },
        {
          "subpart_number"

final Ans

**By Qwen**

In [93]:
prompt_final=f"just tell me the topic and araange them according to obtained marks in ascending order"

In [94]:
messages = [
    {
        "role": "user",
        "content": [
            #{"type": "image", "image": image_path},
            {"type": "text", "text": prompt_final},
        ],
    }
]

In [95]:
# Preparation for inference
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)

In [96]:
inputs = inputs.to("cuda")

# Inference: Generation of the output
generated_ids = model.generate(**inputs, max_new_tokens=512)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

AttributeError: 'GenerativeModel' object has no attribute 'generate'

In [53]:
# Joining the list into a properly formatted string
formatted_output = "\n".join(output_text).strip()

Answer_sheet_json="Extracted Text:\n" + formatted_output

In [54]:
print(Answer_sheet_json)

Extracted Text:
Sure, here are the questions sorted according to the obtained marks in ascending order:

1. **Q4**
   - Description: Explain memory reference and register reference instructions with suitable examples.
   - Max Marks: 4
   - Marks Obtained: 0

2. **Q2**
   - Description: With the help of flow diagram discuss instruction execution cycle.
   - Max Marks: 5
   - Marks Obtained: 2

3. **Q3**
   - Description: Present a report on the use of common bus system for data transfer? Illustrate data transfer using common bus system through example.
